In [1]:
import random

import requests
from bs4 import BeautifulSoup

# 替换这里的URL为你想提取链接的网页地址
url_template = ''
urls = [
    'https://tjj.sh.gov.cn/ydsj56/index.html'
]
for i in range(1,9):
    urls.append(f'https://tjj.sh.gov.cn/ydsj56/index_{i}.html')


urls

['https://tjj.sh.gov.cn/ydsj56/index.html',
 'https://tjj.sh.gov.cn/ydsj56/index_1.html',
 'https://tjj.sh.gov.cn/ydsj56/index_2.html',
 'https://tjj.sh.gov.cn/ydsj56/index_3.html',
 'https://tjj.sh.gov.cn/ydsj56/index_4.html',
 'https://tjj.sh.gov.cn/ydsj56/index_5.html',
 'https://tjj.sh.gov.cn/ydsj56/index_6.html',
 'https://tjj.sh.gov.cn/ydsj56/index_7.html',
 'https://tjj.sh.gov.cn/ydsj56/index_8.html']

In [2]:
import time
from random import randint
from tqdm.notebook import tqdm

# 设置请求头，模仿浏览器行为
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

links = []
# 发送HTTP请求获取网页内容，这次加入了headers参数
for url in tqdm(urls):
    response = requests.get(url, headers=headers)
    
    # 为了正确处理网页中的中文，需要获取网页的真实编码然后据此解码
    response.encoding = response.apparent_encoding
    
    # 使用BeautifulSoup解析网页内容
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 查找第一个ul标签
    ul_tag = soup.find('ul')
    if ul_tag:
        # 在这个ul标签内查找所有的li标签
        li_tags = ul_tag.find_all('li')
        for li in li_tags:
            a_tag = li.find('a')
            if a_tag and 'href' in a_tag.attrs:
                links.append(a_tag['href'])
    
    sleep_time = randint(1, 3)
    print(f"Sleeping for {sleep_time:.2f} seconds...")
    time.sleep(sleep_time)


  0%|          | 0/9 [00:00<?, ?it/s]

Sleeping for 2.00 seconds...
Sleeping for 2.00 seconds...
Sleeping for 1.00 seconds...
Sleeping for 2.00 seconds...
Sleeping for 2.00 seconds...
Sleeping for 2.00 seconds...
Sleeping for 1.00 seconds...
Sleeping for 2.00 seconds...
Sleeping for 1.00 seconds...


In [3]:
base_url = 'https://tjj.sh.gov.cn'
full_links = [base_url+ link for link in links]

In [4]:
full_links

['https://tjj.sh.gov.cn/ydsj56/20240227/6da1265d10594842a4243e4b37db2fd2.html',
 'https://tjj.sh.gov.cn/ydsj56/20240122/e66b8eaff4734885bda49d5bb7d2b195.html',
 'https://tjj.sh.gov.cn/ydsj56/20231219/d11f5c3cc5494d92a1810366f670c5fa.html',
 'https://tjj.sh.gov.cn/ydsj56/20231116/0b214e319e3049b48d3792938796427e.html',
 'https://tjj.sh.gov.cn/ydsj56/20231018/6eeaed71cfb348be81e208bd1b620fc4.html',
 'https://tjj.sh.gov.cn/ydsj56/20230919/5f03aa1f236447d8b28ae823fcd0697f.html',
 'https://tjj.sh.gov.cn/ydsj56/20230823/35cbdf96e8b04f7f8074ae8e5613afcb.html',
 'https://tjj.sh.gov.cn/ydsj56/20230720/c6d9f96e77e24c28a93c279c85693285.html',
 'https://tjj.sh.gov.cn/ydsj56/20230621/52ac5ead85e34ec58b09683ace0f9817.html',
 'https://tjj.sh.gov.cn/ydsj56/20230519/af5b02ea15a844f9aaa5d632f9991f8f.html',
 'https://tjj.sh.gov.cn/ydsj56/20230421/890055e09ecf4944a0cf3a7bb87e4560.html',
 'https://tjj.sh.gov.cn/ydsj56/20230321/da29d6163d514f67bb19aa94c2c62c0c.html',
 'https://tjj.sh.gov.cn/ydsj56/20230227/

In [5]:
import pandas as pd
data = pd.DataFrame(columns=['sum_entry', 'global_entry', 'hkmo_entry', 'tw_entry'])

In [7]:
from datetime import datetime, timedelta, date
import pandas as pd
import random
from tqdm.notebook import tqdm

def clean_and_convert_to_int_mul(s,mul):
    # 移除字符串中的特殊空格和其他非数字字符，保留小数点
    cleaned = s.replace('\xa0', '').strip()
    # 将清理后的字符串转换为浮点数，然后转换为整数
    try:
        value = int(float(cleaned)*mul)
    except ValueError:
        # 如果转换失败，打印错误并返回None或者适当的默认值
        print(f"Cannot convert '{s}' to int.")
        value = None  # 或者根据需要返回默认值，比如0
    return value

def data_extractor(webpage:BeautifulSoup):
    row_data = []
    
    tbody = webpage.find('tbody')
    all_tr = tbody.find_all('tr')
    table_in_td = None
    for table in soup.find_all('table'):
        if table.find_parent('tr'):
            table_in_td = table
            break
    if table_in_td:
        trs = table_in_td.find_all('tr')
        if len(trs) == 6:
            all_tr = table_in_td.find('tbody').find_all('tr')
            for tr in all_tr[2:6]:
                # 在每个tr中找到第二个td
                td = tr.find_all('td')[1] if len(tr.find_all('td')) > 1 else None
                # 从td中提取span内的数据
                if td:
                    p = td.find('p')
                    if p:
                        row_data.append(clean_and_convert_to_int_mul(p.text,10000))
                    else:
                        print("warning, no p found")
                        row_data.append('')
            return row_data
        all_tr = table_in_td.find('tbody').find_all('tr')
        for tr in all_tr[1:5]:
            # 在每个tr中找到第二个td
            td = tr.find_all('td')[1] if len(tr.find_all('td')) > 1 else None
            # 从td中提取span内的数据
            if td:
                p = td.find('p')
                if p:
                    row_data.append(clean_and_convert_to_int_mul(p.text,10000))
                else:
                    print("warning, no p found")
                    row_data.append('')
        return row_data
    elif len(all_tr) <= 6: # version 1
        # 限制范围以防超出tr_tags的长度            
        for tr in all_tr[1:5]:
            # 在每个tr中找到第二个td
            td = tr.find_all('td')[1] if len(tr.find_all('td')) > 1 else None
            # 从td中提取span内的数据
            if td:
                span = td.find('span')
                p = td.find('p')
                if span:
                    row_data.append(clean_and_convert_to_int_mul(span.text,10000))
                elif p:
                    row_data.append(clean_and_convert_to_int_mul(p.text,10000))
                else:
                    print("warning, no p found")
                    row_data.append('')
        return row_data
    #version3,4 tr中无表, version4处理器2020.10.1特殊情况
    else:
        ind = all_tr[0].find('td').find('p').text.strip()
        start = 3
        end =7
        if ind == "":
            start =2
            end =6
        for tr in all_tr[start:end]:
            # 在每个tr中找到第二个td
            td = tr.find_all('td')[1] if len(tr.find_all('td')) > 1 else None
            # 从td中提取span内的数据
            if td:
                p = td.find('p')
                if p:
                    span = p.find('span')
                    if span:
                        row_data.append(clean_and_convert_to_int_mul(span.text,10000))
                    else:
                        row_data.append(clean_and_convert_to_int_mul(p.text,10000))
                elif not td.find(): # no child
                    row_data.append(clean_and_convert_to_int_mul(td.text,10000))
                else:
                    print("warning, no span found")
                    row_data.append('')
        return row_data
    

# 初始化空的DataFrame和起始时间
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
    # 更多用户代理字符串...
]

# 定义请求函数，包括重试逻辑
def make_request_with_retry(url, max_retries=3):
    retry_count = 0
    while retry_count < max_retries:
        # 随机选择一个用户代理
        user_agent = random.choice(user_agents)
        headers = {'User-Agent': user_agent}
        
        try:
            response = requests.get(url, headers=headers)
            # 检查响应代码，如果不是408，则返回响应
            if response.status_code != 408:
                return response
            else:
                print(f"请求超时，尝试次数 {retry_count + 1}/{max_retries}")
        except requests.exceptions.RequestException as e:
            print(f"请求过程中发生错误：{e}")
        
        retry_count += 1
        time.sleep(1)  # 在重试之间稍微等待一段时间，可以根据需要调整
    
    return None  # 所有重试尝试后仍然失败

skip = 1 #2021-10-01，2018-02-01，2014-04-01，2011-10-01
current_position = data.shape[0]+skip

# 多次爬取，为中途程序出错做打算
for i in tqdm(range(current_position,len(full_links))):
    response = make_request_with_retry(full_links[i], max_retries=3)
    # 为了正确处理网页中的中文，需要获取网页的真实编码然后据此解码
    response.encoding = response.apparent_encoding
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    row_data = data_extractor(soup)
    ym = full_links[i].replace("https://","").split('/')[2]
    ym = (datetime.strptime(ym, '%Y%m%d').replace(day=1) - timedelta(days=1)).replace(day=1)
    ym = date(day=ym.day,month=ym.month,year=ym.year)
    # 检查row_data长度，确保它包含4个元素
    if len(row_data) == 4:
        # 将这四个数据点作为一行添加到DataFrame中
        data.loc[ym] = row_data
        
    # sleep_time = random.uniform(1,2)
    # time.sleep(sleep_time)
    print(f'time:{ym},data:{row_data}')

  0%|          | 0/129 [00:00<?, ?it/s]

time:2021-09-01,data:[75000, 41600, 12800, 20600]
time:2021-08-01,data:[76100, 43099, 12100, 20900]
time:2021-07-01,data:[112400, 60100, 23900, 28400]
time:2021-06-01,data:[96500, 52699, 19700, 24100]
time:2021-05-01,data:[101500, 54900, 20700, 25900]
time:2021-04-01,data:[98100, 54900, 19100, 24100]
time:2021-03-01,data:[91300, 48600, 16000, 26700]
time:2021-02-01,data:[58200, 35200, 7900, 15100]
time:2021-01-01,data:[68600, 39300, 11299, 18000]
time:2020-12-01,data:[90300, 54700, 15100, 20500]
time:2020-11-01,data:[88900, 51800, 15000, 22100]
time:2020-10-01,data:[100900, 56600, 16600, 27700]
time:2020-09-01,data:[92600, 51100, 14700, 26800]
time:2020-08-01,data:[89500, 47800, 14200, 27500]
time:2020-07-01,data:[68600, 37300, 10200, 21100]
time:2020-06-01,data:[53099, 30099, 6500, 16500]
time:2020-06-01,data:[46900, 24900, 5800, 16200]
time:2020-04-01,data:[17000, 8600, 2100, 6300]
time:2020-03-01,data:[66000, 42000, 3800, 20200]
time:2020-02-01,data:[82500, 48099, 6200, 28200]
time:

# 爬取结束，手动补充缺失值

In [17]:
data_bak = data.copy()

In [16]:
#2021-10-01，2018-02-01，2014-04-01，2011-10-01
data.loc[date(year=2021,month=10,day=1)]=[101100,54300,19600,27200]
data.loc[date(year=2018,month=2,day=1)]=[470600,335200,46200,89200]
data.loc[date(year=2014,month=4,day=1)]=[752200,586500,65600,100100]
data.loc[date(year=2011,month=10,day=1)]=[777400,622700,61600,93100]
data.loc[date(year=2011,month=12,day=1)]=[610500,486700,51600,72200]
# add data where originally does not exist
# 2022-02-01,72447,36222,12587,22844
# 2022-03-01,70797,30970,13812,24392
# 2022-04-01,5626,3310,984,1280
# 2022-05-01,10629,6383,2073,2091
data.loc[date(year=2022,month=5,day=1)]=[10629,6383,2073,2091]
data.loc[date(year=2022,month=4,day=1)]=[5626,3310,984,1280]
data.loc[date(year=2022,month=3,day=1)]=[70797,30970,13812,24392]
data.loc[date(year=2022,month=2,day=1)]=[72447,36222,12587,22844]

/var/folders/wb/86d7j5hx6lz71vnrk3q5ksmr0000gn/T/ipykernel_46924/43340782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[date(year=2021,month=10,day=1)]=[101100,54300,19600,27200]
/var/folders/wb/86d7j5hx6lz71vnrk3q5ksmr0000gn/T/ipykernel_46924/43340782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[date(year=2018,month=2,day=1)]=[470600,335200,46200,89200]
/var/folders/wb/86d7j5hx6lz71vnrk3q5ksmr0000gn/T/ipykernel_46924/43340782.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [21]:
import pandas as pd

# 假设你的DataFrame是df，并且索引已经是datetime类型
# 如果不是，你需要先转换索引
data = data[~data.index.duplicated(keep='first')]
data.index = pd.to_datetime(data.index)

# 创建一个起始日期和结束日期之间的完整月份日期序列
# 假设起始日期和结束日期分别是你的DataFrame中的最小和最大日期
start_date = data.index.min()
end_date = data.index.max()
full_date_range = pd.date_range(start=start_date, end=end_date, freq='MS') # 'MS' 代表月初

# 使用 reindex 补全缺失的日期
data = data.reindex(full_date_range)

# 查看结果
data

,sum_entry,global_entry,hkmo_entry,tw_entry
2011-01-01,541600,440000,42500,59100
2011-02-01,572700,432800,45700,94200
2011-03-01,683400,548100,56900,78400
2011-04-01,706300,551900,65400,89000
2011-05-01,706500,559400,56600,90500
...,...,...,...,...
2023-09-01,375600,269700,37600,68300
2023-10-01,423200,291700,45800,85700
2023-11-01,477700,343800,52300,81600
2023-12-01,444900,315700,61600,67600


In [25]:
data.loc[datetime.strptime("2020-01-01","%Y-%m-%d")] = [489800,377100,43000,69700]
data.loc[datetime.strptime("2020-05-01","%Y-%m-%d")] =[46900,24900,5800,16200]
data.loc[datetime.strptime("2020-06-01","%Y-%m-%d")] =[53100,30100,6500,16500]

data

,sum_entry,global_entry,hkmo_entry,tw_entry
2011-01-01,541600,440000,42500,59100
2011-02-01,572700,432800,45700,94200
2011-03-01,683400,548100,56900,78400
2011-04-01,706300,551900,65400,89000
2011-05-01,706500,559400,56600,90500
...,...,...,...,...
2023-09-01,375600,269700,37600,68300
2023-10-01,423200,291700,45800,85700
2023-11-01,477700,343800,52300,81600
2023-12-01,444900,315700,61600,67600


In [27]:
df_links = pd.DataFrame(full_links,columns=['url'])
df_links

,url
0,https://tjj.sh.gov.cn/ydsj56/20240227/6da1265d...
1,https://tjj.sh.gov.cn/ydsj56/20240122/e66b8eaf...
2,https://tjj.sh.gov.cn/ydsj56/20231219/d11f5c3c...
3,https://tjj.sh.gov.cn/ydsj56/20231116/0b214e31...
4,https://tjj.sh.gov.cn/ydsj56/20231018/6eeaed71...
...,...
148,https://tjj.sh.gov.cn/ydsj56/20110606/0014-235...
149,https://tjj.sh.gov.cn/ydsj56/20110508/0014-235...
150,https://tjj.sh.gov.cn/ydsj56/20110406/0014-235...
151,https://tjj.sh.gov.cn/ydsj56/20110307/0014-235...


In [28]:
from sqlalchemy import create_engine

database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

data.to_sql('sh_customs', con=engine, if_exists='replace', index=True)
df_links.to_sql('sh_links', con=engine, if_exists='replace', index=True)

153